In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

%matplotlib inline

In [2]:
# Get the data from the csv file.
def getData(filename):
    with open(filename) as file:
        rows = file.readlines()
        labels = []
        images = []
        for row in rows[1:]:
            label = np.array(row[0]).astype(float)
            labels.append(label)
            image = row.split(',')
            image = np.array(image).astype(float)
            image = np.array_split(image[1:], 28)
            images.append(image)
        labels = np.array(labels)
        images = np.array(images)
    return images, labels

train_path = 'sign_language/sign_mnist_train/sign_mnist_train.csv'
test_path = 'sign_language/sign_mnist_test/sign_mnist_test.csv'
train_images, train_labels = getData(train_path)
test_images, test_labels = getData(test_path)


KeyboardInterrupt: 

In [ ]:
# Norrmalize pixels
train_images = train_images/255
test_images = test_images/255

# Expand dimension to 28 x 28 x 1
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

In [ ]:
# Just want to see what the pixel looks like.
def seeImage(img_pixels, label):
    counter = dict()

    img_pixels = np.array(img_pixels, dtype='uint8')
    image = Image.fromarray(img_pixels)

    filename = '{}{}.png'.format(label, "_test")
    image.save(filename)
    img = cv2.imread(filename, 0)
    plt.imshow(img, cmap='gray')
    
    return img

#seeImage(train_images[0], train_labels[0])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Image Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2, 
    zoom_range=0.3,
    fill_mode='nearest',
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator()

In [7]:
from keras.utils.np_utils import to_categorical

train_labels_cat = to_categorical(train_labels, 24)
test_labels_cat = to_categorical(test_labels, 24)

#print(train_labels_cat[0])

Using TensorFlow backend.


In [18]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

# Model training using CNN
def modelCnn():
    model = Sequential()

    model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Conv2D(filters=64, kernel_size=(3,3),activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=128, kernel_size=(2,2), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))

    model.add(Flatten())    
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(24, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

model = modelCnn()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 25, 25, 32)        544       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 128)         32896     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0         
__________

In [19]:
# Training the model

trained_model = model.fit_generator(train_datagen.flow(train_images, train_labels_cat, batch_size=32),
                                    steps_per_epoch=len(train_images) / 32,
                                    epochs=10,
                                    validation_data=validation_datagen.flow(test_images, test_labels_cat, batch_size=32),
                                    validation_steps=len(test_images) / 32)

Epoch 1/20
858/857 [==============================] - 13s 15ms/step - loss: 1.5938 - acc: 0.4620 - val_loss: 1.3048 - val_acc: 0.4297
Epoch 2/20
858/857 [==============================] - 12s 14ms/step - loss: 1.3199 - acc: 0.5031 - val_loss: 0.8635 - val_acc: 0.6654
Epoch 3/20
858/857 [==============================] - 12s 14ms/step - loss: 1.0903 - acc: 0.5764 - val_loss: 0.6409 - val_acc: 0.7423
Epoch 4/20
858/857 [==============================] - 13s 15ms/step - loss: 0.9350 - acc: 0.6317 - val_loss: 0.5479 - val_acc: 0.7748
Epoch 5/20
858/857 [==============================] - 13s 15ms/step - loss: 0.8146 - acc: 0.6858 - val_loss: 0.4656 - val_acc: 0.8148
Epoch 6/20
858/857 [==============================] - 13s 15ms/step - loss: 0.7220 - acc: 0.7220 - val_loss: 0.4350 - val_acc: 0.8410
Epoch 7/20
858/857 [==============================] - 13s 16ms/step - loss: 0.6507 - acc: 0.7516 - val_loss: 0.3294 - val_acc: 0.8802
Epoch 8/20
858/857 [==============================] - 13s 15ms

KeyboardInterrupt: 

In [13]:
len(train_images/32)

27455